## Data Loading: TensorFlow TFRecord
 This example demonstrates how to utilise rocAL to access data that has been saved in the TensorFlow TFRecord format.

## Common Code

In [ ]:
from amd.rocal.plugin.tf import ROCALIterator
from amd.rocal.pipeline import Pipeline
import amd.rocal.types as types
import amd.rocal.fn as fn
import tensorflow as tf
import os
import matplotlib.pyplot as plt
%matplotlib inline

## Configuring rocAL pipeline

Configure the pipeline parameters as required by the user.

In [ ]:
image_path = os.path.join(os.environ['ROCAL_DATA_PATH'], "tf", "classification")
rocal_cpu = True
batch_size = 10
one_hot_label = 0
num_threads = 1
local_rank = 1
tf_record_reader_type = 0
featureKeyMap = {
    "image/encoded": "image/encoded",
    "image/class/label": "image/class/label",
    "image/filename": "image/filename",
}
path = "OUTPUT_FOLDER/TF_READER/CLASSIFICATION/"

## Defining and Running the Pipeline
 Creating the pipeline using tf.readers for reading the images stored in tf_record format. In this pipeline, resize augmentation is used on the decoded images.<br>Resize augmentation outputs are returned using set_outputs

In [ ]:
pipe = Pipeline(batch_size=batch_size, num_threads=num_threads, device_id=local_rank, seed=2, rocal_cpu=rocal_cpu, tensor_layout = types.NCHW)


In [ ]:
with pipe:
        inputs = fn.readers.tfrecord(path=image_path, reader_type=tf_record_reader_type, user_feature_key_map=featureKeyMap,
            features={
                "image/encoded": tf.io.FixedLenFeature((), tf.string, ""),
                "image/class/label": tf.io.FixedLenFeature([1], tf.int64, -1),
                "image/filename": tf.io.FixedLenFeature((), tf.string, "")
            }
        )
        jpegs = inputs["image/encoded"]
        images = fn.decoders.image(jpegs, user_feature_key_map=featureKeyMap, output_type=types.RGB, path=image_path)
        resized = fn.resize(images, resize_width=300, resize_height=300)
        if(one_hot_label == 1):
            labels = inputs["image/class/label"]
            _ = fn.one_hot(labels, num_classes=1000)
        pipe.set_outputs(resized)

In [ ]:
pipe.build()
# Dataloader
imageIterator = ROCALIterator(pipe)

## Visualizing outputs
We have displayed the output of augmented images using imshow()

In [ ]:
cnt = 0
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(20, 20))
row = 0
col = 0
for i, ([images_array], labels_array) in enumerate(imageIterator, 0):
    for k in images_array:
        cnt += 1
        image = k.astype("int")
        axes[row, col].imshow(image)
        row += 1
        if row == 2:
            row = 0
            if col != 4:
                col += 1
imageIterator.reset()